In [1]:
def num_flat_features(x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features

In [2]:
import torch
import torch.nn as nn

class NormalizationLayer(nn.Module):

    def __init__(self):
        super(NormalizationLayer, self).__init__()

    def forward(self, x, epsilon=1e-8):
        return x / torch.sqrt(torch.mean(x ** 2, dim=1, keepdim=True) + 1e-8)


C:\Users\micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class AudioNorm(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return input / torch.sqrt(torch.mean(input ** 2, dim=1, keepdim=True)
                                  + 1e-8)


In [10]:
class WSConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, gain=2) -> None:
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.scale = (gain / (in_channels * kernel_size**2)) ** 0.5
        self.bias = self.conv.bias
        self.conv.bias = None # remove bias

        nn.init.normal_(self.conv.weight)
        nn.init.zeros_(self.bias)
        

    def forward(self, z):
        return self.conv(z * self.scale) + self.bias.view(1, self.bias.shape[0], 1, 1)

In [48]:
# initial = nn.Sequential(
#     # AudioNorm(),
#     nn.ConvTranspose2d(256, 256, 4, 1, 0),
#     nn.LeakyReLU(0.2),
#     nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
#     nn.LeakyReLU(0.2),
# )

initial1 = nn.Sequential(
    nn.ConvTranspose2d(256, 256, 4, 1, 0),
    nn.LeakyReLU(0.2),
)

initial2 = nn.Sequential(
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
    nn.LeakyReLU(0.2),
)

deconv = nn.ConvTranspose2d(
    in_channels=256, 
    out_channels=256, 
    kernel_size=(8, 5), 
    stride=1, 
    padding=0
)

initial_rgb = WSConv2d(
    in_channels=256,
    out_channels=1,
    kernel_size=1,
    stride=1,
    padding=0
)

In [49]:
z = torch.randn(64, 256, 1, 1)
z.shape

torch.Size([64, 256, 1, 1])

In [50]:
z = deconv(z)
print(z.shape)
z = initial2(z)
print(z.shape)

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (8 x 5). Kernel size can't be greater than actual input size